## Inference Demo with Pre-trained Models: Python.


### Create the virtual environment
Install the conda environment before running this notebook. 
```bash
conda env create -f environment.yml
conda activate detectron2
```

### Install the Detectron2 for MacOS (CPU)

```bash
git clone https://github.com/facebookresearch/detectron2.git
```

Open `detectron2/detectron2/config/defaults.py` change the line 28:
```python
...
# _C.MODEL.DEVICE = "cuda"
_C.MODEL.DEVICE = "cpu"
...
```

Then in the root foler run the following command to install the pacakge:
```base
CC=clang CXX=clang++ python -m pip install -e detectron2
```



##  Instance Segmentation: Image

In [ ]:
# Some basic setup:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
# Setup detectron2 logger
import detectron2
print(detectron2.__version__)

from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import random

In [ ]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
from matplotlib import pyplot as plt
import cv2
sample_image_path = './data/MVD_research_samples/Oceania/kdbqeR8gwZlS-gEXQMWo_g.jpg'
im = cv2.cvtColor(cv2.imread(sample_image_path), cv2.COLOR_BGR2RGB)
plt.figure(figsize=[12.8, 9.6])
plt.imshow(im)

In [ ]:
config_model_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
threshold = 0.1
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file(config_model_name))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_model_name)
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
print(outputs["instances"].scores)
print(outputs["instances"].pred_classes)
# print(outputs["instances"].pred_boxes)

In [ ]:
v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
im_out = out.get_image()

In [ ]:
plt.figure(figsize=[12.8, 9.6])
plt.imshow(im_out)
cv2.imwrite('results/output-InstanceSegmentation.png', cv2.cvtColor(im_out, cv2.COLOR_RGB2BGR))

###  Panoptic Segmentation: Image

In [ ]:
# Inference with a panoptic segmentation model
config_model_name = "COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(config_model_name))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_model_name)
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]

v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)

In [ ]:
im_out = out.get_image()
plt.figure(figsize=[12.8, 9.6])
plt.imshow(im_out)
cv2.imwrite('results/output-PanopticSegmentation.png', cv2.cvtColor(im_out, cv2.COLOR_RGB2BGR))

## Inference Image Demo with Pre-trained Models: Command Line.

Pick a model and its config file from model zoo, for example, mask_rcnn_R_50_FPN_3x.yaml.
Detectron2 provides the `demo.py` that is able to run builtin standard models. Run it with:

```bash
!python detectron2/demo/demo.py --config-file detectron2/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml  \
  --input input1.jpg input2.jpg \
  [--other-options]
  --opts MODEL.WEIGHTS detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl

```
To repeat the above python command

### COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml

In [ ]:
!python detectron2/demo/demo.py \
--config-file detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml \
--input data/MVD_research_samples/Oceania/kdbqeR8gwZlS-gEXQMWo_g.jpg \
--confidence-threshold 0.1 \
--output results/output-InstanceSegmentation-cmd.png \
--opts MODEL.WEIGHTS detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl

### COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml

In [ ]:
!python detectron2/demo/demo.py \
--config-file detectron2/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml \
--input data/MVD_research_samples/Oceania/kdbqeR8gwZlS-gEXQMWo_g.jpg \
--confidence-threshold 0.6 \
--output results/output-PanopticSegmentation-cmd.png \
--opts MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl

## Inference Video Demo with Pre-trained Models: Command Line.

Model Type: COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml

This 1s video clip may cost 0.5 hour to run on MacOS CPU. Here is just a demo to prove it works. 

In [ ]:
!python detectron2/demo/demo.py \
--config-file detectron2/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml \
--video-input data/video/video-clip-1s.mp4 \
--confidence-threshold 0.6 \
--output results/video-output.mkv \
--opts MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl